In [1]:
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
from matplotlib.pyplot import plot

calls_service = pd.read_csv('mfdems.csv') # EMS Calls for Service Milwaukee
liq_lic = pd.read_csv('liquorlicenses.csv') # Liquor License data Milwaukee
## https://www.irs.gov/statistics/soi-tax-stats-individual-income-tax-statistics-zip-code-data-soi
nat_irs_16 = pd.read_csv('16zpallagi.csv') # IRS Data 2016
nat_irs_16['zipcode'] = nat_irs_16['zipcode'].astype(np.float64)

In [2]:
mke_zip = [x for x in calls_service['ZIP Code'].unique() if ~np.isnan(x)]
mke_ald = [x for x in calls_service['Aldermanic District'].unique() if ~np.isnan(x)]
mke_irs_16 = nat_irs_16[nat_irs_16['zipcode'].isin(mke_zip)]

In [3]:
n_one_mke = mke_irs_16[['zipcode','agi_stub','N1']].groupby(['zipcode','agi_stub']).agg({'N1':['sum',lambda x: 100*x.sum()/(mke_irs_16['N1'].sum())]})['N1']
#n1_mke.columns.str.replace('<lambda>','% of total pop')
n_one_mke_clean = n_one_mke[n_one_mke['sum'] > 0]
n_one_mke = n_one_mke_clean.sort_values(by=['agi_stub','sum'],ascending=False)
n_one_mke.head()

,,sum,<lambda>
zipcode,agi_stub,,
53217.0,6,2760,0.624632
53211.0,6,1560,0.353053
53051.0,6,1320,0.298737
53132.0,6,1100,0.248948
53202.0,6,990,0.224053


In [4]:
agi_foreach_zip = mke_irs_16[['zipcode','agi_stub','N1']].pivot(index='agi_stub',columns='zipcode',values='N1')
agi_foreach_zip

zipcode,53051.0,53110.0,53130.0,53132.0,53154.0,53172.0,53202.0,53203.0,53204.0,53205.0,...,53220.0,53221.0,53222.0,53223.0,53224.0,53225.0,53226.0,53227.0,53228.0,53233.0
agi_stub,,,,,,,,,,,,,,,,,,,,,
1,5240,3310,1220,4750,5240,3720,3770,290,8920,2170,...,4760,7020,3900,5290,3950,5180,2530,4010,2440,1960
2,3810,2680,940,3140,4150,2830,3990,310,4930,820,...,4180,5190,3520,3680,2480,3520,2010,3850,1910,670
3,2820,1550,670,2440,2940,1660,2720,220,1130,230,...,2360,2890,2270,2030,1150,1380,1650,2170,1240,150
4,2290,920,470,2120,2080,1080,1190,100,330,80,...,1320,1700,1360,1110,700,740,1150,1210,930,40
5,3880,770,730,3880,3170,1100,1610,120,240,90,...,1270,1650,1430,1080,850,590,1890,1060,1220,40
6,1320,60,120,1100,420,100,990,90,110,0,...,110,150,140,120,100,50,650,100,230,0


In [5]:
agi_by_zip = mke_irs_16[['zipcode','agi_stub','N1']].pivot(index='zipcode',columns='agi_stub',values='N1')
pop_by_zip = mke_irs_16[['zipcode','N1']].groupby(['zipcode']).agg(['sum'])['N1']
errythang = pop_by_zip.join(agi_by_zip)
errythang.head()

,sum,1,2,3,4,5,6
zipcode,,,,,,,
53051.0,19360,5240,3810,2820,2290,3880,1320
53110.0,9290,3310,2680,1550,920,770,60
53130.0,4150,1220,940,670,470,730,120
53132.0,17430,4750,3140,2440,2120,3880,1100
53154.0,18000,5240,4150,2940,2080,3170,420


In [6]:
df=calls_service
df['Incident Date'] = pd.to_datetime(df['Incident Date'])
df['Incident Date'] = df['Incident Date'].dt.strftime('%Y-%m')
call_types = [x for x in df['Final Call for Service Type'].unique()]
call_types.sort()
print(call_types)

['ALARM', 'APPL', 'AUTOF', 'AUTOX', 'Abdominal Pain/Problems', 'Academy - Medical', 'Allergies (Reactions)/Envenomations (Stings, Bites)', 'Animal Bites/Attacks', 'Assault/Sexual Assault', 'Auto Accident', 'Auto Accident - Multiple Injuries', 'BTB1', 'BTB2', 'BTB3', 'Back Pain (Non-Traumatic or Non-Recent Trauma)', 'Bariatric Patient', 'Breathing Problems', 'Burns (Scalds)/Explosion (Blast)', 'CARBON', 'CTB1', 'CTB2', 'CTB3', 'Carbon Monoxide/Inhalation/Hazmat/CBRN', 'Cardiac or Respiratory Arrest/Death', 'Chest Pain (Non-Traumatic)', 'Choking', 'Combative Patient', 'Community Paramedic', 'Convulsions/Seizures', 'DIVE', 'Diabetic Problems', 'Downgraded Response', 'Drowning (Near)/Diving/Scuba Accident', 'ELECT', 'ELEV', 'ER', 'Electrocution/Lightning', 'Eye Problems/Injuries', 'FBRESP', 'FLUSH', 'FORCE', 'FULL', 'FULL2', 'FULL3', 'FULL4', 'Falls', 'Fire Station', 'GARAGE', 'GAS', 'GMIA2', 'GMIA3', 'GRASS', 'HAZMAT', 'HEAT', 'HIRISE1', 'HIRISE3', 'HRRESP', 'HURT', 'HURTRESP', 'Headache'

In [7]:
df.groupby('Original Call for Service Type').agg(['count'])['Final Call for Service Type'].sort_values(by='count',ascending=False).head()

,count
Original Call for Service Type,
Sick Person (Specific Diagnosis),18346
Breathing Problems,12116
Unknown Problem (Man Down),8604
Falls,7400
Traffic/Transportation Accidents,7211


In [8]:
all_calls_by_zip = df[["ZIP Code","Original Call for Service Type"]].groupby(['ZIP Code'], as_index=True).agg(['count']).sort_values(by='ZIP Code', ascending=True)['Original Call for Service Type']
all_calls_by_zip.columns = ['ServiceCalls']
sick_byZip = df[df['Original Call for Service Type']=='Sick Person (Specific Diagnosis)'].groupby('ZIP Code',as_index=True).agg(['count'])['Original Call for Service Type']
sick_byZip.columns=['SickPerson']

service_byZip = all_calls_by_zip.join(sick_byZip)
#
errythang.sort_values(by='zipcode')
service_byZip.sort_values(by= 'ZIP Code')
errythang = errythang.merge(service_byZip, left_index=True, right_index=True, how='left')
errythang.sort_values(by=['ServiceCalls','SickPerson'], ascending=False)

,sum,1,2,3,4,5,6,ServiceCalls,SickPerson
zipcode,,,,,,,,,
53206.0,8380,5790,2000,390,120,80,0,8945,1595.0
53209.0,19900,9860,5340,2180,1120,1170,230,8527,1511.0
53218.0,16670,8980,5100,1610,610,370,0,7854,1368.0
53204.0,15660,8920,4930,1130,330,240,110,6834,1152.0
53212.0,12930,6560,3770,1360,570,520,150,6725,1218.0
53216.0,13580,6950,4050,1440,640,460,40,6632,1136.0
53215.0,24440,12170,8210,2580,910,540,30,6065,940.0
53208.0,12570,6340,3420,1390,630,680,110,5997,1064.0
53210.0,10650,5900,2850,950,420,490,40,5966,1075.0


In [9]:
# MKE Service Call data
cardiac = df[df['Final Call for Service Type']=='Cardiac or Respiratory Arrest/Death'].groupby(['Aldermanic District'], as_index=True).agg(['count'])['Final Call for Service Type']
cardiac.columns = ['CardiacOrRespiratoryArrest/Death']
downgraded = df[df['Final Call for Service Type']=='Downgraded Response'].groupby(['Aldermanic District'], as_index=False).agg(['count'])['Final Call for Service Type']
downgraded.columns = ['DowngradedResponse']
#### Assault is only organized by Aldermanic District; can't get it by ZIP Code
assault_byAldDist = df[df['Final Call for Service Type']=='Assault/Sexual Assault'].groupby('Aldermanic District',as_index=True).agg(['count'])['Final Call for Service Type']
assault_byAldDist.columns=['Assault/SexualAssault']
any_service_call = df[['Aldermanic District','Final Call for Service Type']].groupby(['Aldermanic District'], as_index=True).agg(['count'])['Final Call for Service Type']
any_service_call.columns = ['ServiceCalls']
## MKE Liquor License data
liq_avail = liq_lic[['ALDERMANIC_DISTRICT','EXP_DATE']].groupby(['ALDERMANIC_DISTRICT'], as_index=True).agg(['count']).sort_values(by='ALDERMANIC_DISTRICT', ascending=True)['EXP_DATE']
liq_avail.columns=['LiquorLicenses']

In [18]:
any_card_down = any_service_call.join(cardiac.join(downgraded)).join(assault_byAldDist)
all_byAldDist = any_card_down.merge(liq_avail,left_index=True,right_index=True,how='left')

In [19]:
all_byAldDist.sort_values(by=['ServiceCalls','Assault/SexualAssault','CardiacOrRespiratoryArrest/Death','DowngradedResponse','LiquorLicenses'], ascending=False)

,ServiceCalls,CardiacOrRespiratoryArrest/Death,DowngradedResponse,Assault/SexualAssault,LiquorLicenses
Aldermanic District,,,,,
15.0,12867,96,362,878,58
6.0,11953,82,206,756,99
4.0,11200,86,301,408,269
1.0,10595,68,194,591,35
7.0,10492,81,263,646,30
2.0,8807,77,248,466,22
9.0,8024,106,106,294,32
12.0,7033,81,206,425,188
8.0,5276,68,92,274,68


In [15]:
## Notice that District 4 is in the top five of all three lists!
## That's not a good thing

In [16]:
all_byAldDist.plot(x='LiquorLicenses',y='CardiacRespiratoryArrestDeath', style='o')
all_byAldDist.plot(x='LiquorLicenses',y='DowngradedResponse', style='o')

KeyError: 'CardiacRespiratoryArrestDeath'